In [1]:
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import sys
from keras.models import load_model
import os
import random
import math
import numpy as np
from imutils import paths

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import helpers
from helpers import Audio

In [3]:
audio = Audio(path='audio/data/')
trainlist=audio.generate_audio_list(p='train')

['ABSOLUTELY', 'ABOUT', 'ABUSE']
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]]
['ABSOLUTELY', 'ABOUT', 'ABUSE']


In [4]:
valX,valY = audio.load_audio(p='val')

[INFO] loading images...
['ABSOLUTELY', 'ABOUT', 'ABUSE']
(150,)


In [5]:
part='test'
testX,testY = audio.load_audio(p=part)

[INFO] loading images...
['ABSOLUTELY', 'ABOUT', 'ABUSE']
(150,)


In [6]:
val_Y=valY.toarray()
test_Y = testY.toarray()
#train_Y = trainY.toarray()

In [7]:
#trainX = np.reshape(trainX,(-1,26,121))
valX = np.reshape(valX,(-1,26,121))
testX = np.reshape(testX,(-1,26,121))

In [8]:
from keras.models import Sequential,Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,Conv3D,Conv1D
from keras.layers.convolutional import MaxPooling2D,MaxPooling3D,MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import LSTM,Input,GlobalAveragePooling1D
from keras import backend as K
from keras.optimizers import Adam,SGD
from keras.layers.wrappers import TimeDistributed, Bidirectional
import tensorflow as tf
#from keras.layers.extra import TimeDistributedConvolution2D, TimeDistributedMaxPooling2D, TimeDistributedFlatten

In [9]:
class_num = 3

# mfcc

In [10]:
input_audio = Input(shape = (26,121))

conv1d_1 = Conv1D(64,kernel_size = 3,strides=1, padding = 'same',
               activation = 'relu')(input_audio)

batch_norm1 = BatchNormalization()(conv1d_1)

batch_norm1 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm1)

conv1d_2 = Conv1D(128,kernel_size = 3, strides = 1, padding = 'same',
               activation = 'relu')(batch_norm1)

batch_norm2 = BatchNormalization()(conv1d_2)
batch_norm2 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm2)

conv1d_2 = Conv1D(128,kernel_size = 3, strides = 1, padding = 'same',
               activation = 'relu')(batch_norm2)

batch_norm2 = BatchNormalization()(conv1d_2)
batch_norm2 = MaxPooling1D(pool_size=2,padding = 'same')(batch_norm2)

bilstm1 = Bidirectional(LSTM(256,return_sequences=True))(batch_norm2)
bilstm2 = Bidirectional(LSTM(256,return_sequences=True),name = 'bilstm2')(bilstm1)

avgpool = GlobalAveragePooling1D()(bilstm2)

dense1 = Dense(256,activation='relu')(avgpool)
batch_norm3 = BatchNormalization()(dense1)
dense2 = Dense(class_num,activation = 'softmax')(batch_norm3)
#maxpooling1d = MaxPooling1D(pool_size=2, padding='same')(batch_norm2)



In [11]:
model = Model(inputs=[input_audio], 
              outputs=[dense2])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 26, 121)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 26, 64)            23296     
_________________________________________________________________
batch_normalization_1 (Batch (None, 26, 64)            256       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 13, 128)           24704     
_________________________________________________________________
batch_normalization_2 (Batch (None, 13, 128)           512       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 128)            0         
__________

In [12]:
from keras.utils import plot_model
plot_model(model, to_file='model_audio.png')

In [13]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


In [14]:
type(trainlist)

list

In [ ]:
sample_number = len(trainlist)
model.fit_generator(audio.generate_arrays_from_file(filelist=trainlist),
                        validation_data=(valX, val_Y),
                        steps_per_epoch=sample_number // 64,
                        epochs=5, verbose=1)

Epoch 1/5
46/46 [==============================] - 14s 315ms/step - loss: 0.0133 - acc: 0.9997 - val_loss: 0.0440 - val_acc: 0.9800
Epoch 2/5
46/46 [==============================] - 12s 257ms/step - loss: 0.0103 - acc: 1.0000 - val_loss: 0.0413 - val_acc: 0.9800
Epoch 3/5
31/46 [===================>..........] - ETA: 3s - loss: 0.0081 - acc: 1.0000

In [ ]:
model.fit(trainX, train_Y, epochs=5, batch_size=64, 
          verbose=1, validation_data=(valX, val_Y))

In [ ]:
model.save('try.h5')

In [15]:
xmodel=load_model('try.h5')

In [ ]:
xmodel.summary()

In [20]:
# intermediate_layer_model = Model(inputs=xmodel.input,
#                                  outputs=xmodel.get_layer('bilstm2').output)
# newmodel.compile(loss="categorical_crossentropy", optimizer=sgd,
# 	metrics=["accuracy"])
intermediate = xmodel.get_layer('bilstm2').output
avgpool1 = GlobalAveragePooling1D()(intermediate)

dense11 = Dense(256,activation='relu')(avgpool1)
batch_norm11 = BatchNormalization()(dense11)
dense11 = Dense(64,activation='relu')(batch_norm11)
batch_norm12 = BatchNormalization()(dense11)
dense12 = Dense(class_num,activation = 'softmax')(batch_norm12)
#m = intermediate_layer_model.predict(testX)


In [21]:
model1 = Model(inputs=[xmodel.input], 
              outputs=[dense12])
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 26, 121)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 26, 64)            23296     
_________________________________________________________________
batch_normalization_5 (Batch (None, 26, 64)            256       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 13, 64)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 13, 128)           24704     
_________________________________________________________________
batch_normalization_6 (Batch (None, 13, 128)           512       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 7, 128)            0         
__________

In [22]:
print("[INFO] compiling model...")
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9)
model1.compile(loss="categorical_crossentropy", optimizer=sgd,
	metrics=["accuracy"])

[INFO] compiling model...


In [23]:
sample_number = len(trainlist)
model1.fit_generator(audio.generate_arrays_from_file(filelist=trainlist),
                        validation_data=(valX, val_Y),
                        steps_per_epoch=sample_number // 64,
                        epochs=5, verbose=1)

Epoch 1/5
46/46 [==============================] - 15s 322ms/step - loss: 0.2686 - acc: 0.8940 - val_loss: 0.0744 - val_acc: 0.9733
Epoch 2/5
46/46 [==============================] - 12s 252ms/step - loss: 0.0274 - acc: 0.9966 - val_loss: 0.0596 - val_acc: 0.9800
Epoch 3/5
46/46 [==============================] - 12s 269ms/step - loss: 0.0160 - acc: 1.0000 - val_loss: 0.0560 - val_acc: 0.9800
Epoch 4/5
46/46 [==============================] - 12s 255ms/step - loss: 0.0117 - acc: 1.0000 - val_loss: 0.0535 - val_acc: 0.9800
Epoch 5/5
46/46 [==============================] - 13s 275ms/step - loss: 0.0094 - acc: 1.0000 - val_loss: 0.0517 - val_acc: 0.9800


In [ ]:
model1.fit(trainX, train_Y, epochs=5, batch_size=64, 
          verbose=1, validation_data=(valX, val_Y))

In [16]:
model.evaluate(testX,test_Y)

150/150 [==============================] - 0s 1ms/step


[0.12325750867525737, 0.9666666666666667]